In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
def filter_string(var):
    var = var.replace(" ", "_")
    var = var.lower()
    return var

def sql_to_dataframe(db_tup):
    return pd.DataFrame(data=db_tup[1], columns=db_tup[0])

def create_national_table(df, year):
    with sqlite3.connect("safe_to_delete.db") as conn:
        df.to_sql(f"national_data_{year}", con = conn, if_exists= "replace", index= False)

def create_state_table(df, year):
    with sqlite3.connect("safe_to_delete.db") as conn:
        df.to_sql(f"arizona_state_data_{year}", con = conn, if_exists= "replace", index= False)

def run_query(q):
    with sqlite3.connect("safe_to_delete.db") as conn:# create connection
        cur = conn.cursor() # create cursor object

        cur.execute(q) # create execution

        results = cur.fetchall()
        colnames = [n[0] for n in cur.description]

    return (colnames, results)

def run_commit_query(q):
    with sqlite3.connect("safe_to_delete.db") as conn:# create connection
        cur = conn.cursor() # create cursor object
        cur.execute(q) # create execution
        conn.commit()

def clean_df(df):
    def clean_column(var):
        col = df[var]
        col = col.astype(str)
        col = col.str.replace("$", "")
        col = col.str.replace("(", "")
        col = col.str.replace(")", "")
        col = col.str.replace(",", "")
        return col

    df.columns = [filter_string(word) for word in df.columns] # replaces spaces with underscores and makes every letter lowercase
    df = df.loc[:, ['occupation_title_(click_on_the_occupation_title_to_view_its_profile)', 'employment', 'mean_hourly_wage', 'annual_mean_wage', ]] # constrain the df to these columns
    df.columns = ['occupation_title', 'employment', 'mean_hourly_wage', 'annual_mean_wage', ]
    df = df.drop(df.index[0]) # remove the first row

    df = df.reset_index(drop=True)

    df["mean_hourly_wage"] = clean_column("mean_hourly_wage").astype(float)
    df["annual_mean_wage"] = clean_column("annual_mean_wage").astype(float)
    df["employment"] = clean_column("employment").astype(float)

    return df

def make_plot(x, y, title, xlabel, ylabel):
    fig = plt.figure()
    fig.patch.set_facecolor("#D5E8F3")

    colors = ["red", "blue"] * (len(x) // 2)
    if len(x) % 2:
        colors.append("red")

    plt.scatter(x, y, color = colors)
    plt.title(title)
    plt.xlabel(xlabel, fontsize=8)
    plt.ylabel(ylabel, fontsize=8)
    plt.xticks(rotation=45, ha="right", fontsize=8)
    plt.yticks(np.linspace(min(y), max(y), 10))

    plt.grid(axis="y", color="gray", linewidth=0.5)
    plt.grid(axis="x", color="gray", linewidth=0.5)
    plt.gca().set_facecolor('#E6F1F8')

    for color, label in zip(colors, plt.gca().get_xticklabels()):
        label.set_color(color)
        
    #plt.subplots_adjust(bottom=0.5)

def combine_national_state_df(df_one, df_two):
    combined_df = pd.DataFrame()
    combined_df["occupation_title"] = df_one["occupation_title"]
    combined_df["national_employment"] = df_one["total_employment"]
    combined_df["arizona_state_employment"] = df_two["employment"]
    combined_df = combined_df.dropna()
    #combined_df.loc[8, "occupation_title"] = "Health and Nursing Assistants"
    return combined_df

def create_clean_national_state_dfs():
    global national_2023_df, national_2022_df, national_2021_df, national_2020_df, national_2019_df
    global arizona_2023_df, arizona_2022_df, arizona_2021_df, arizona_2020_df, arizona_2019_df
    # create and clean national and state dataframes
    national_2023_df = clean_df(national_2023_df)
    arizona_2023_df = clean_df(arizona_2023_df)

    national_2022_df = clean_df(national_2022_df)
    arizona_2022_df = clean_df(arizona_2022_df)

    national_2021_df = clean_df(national_2021_df)
    arizona_2021_df = clean_df(arizona_2021_df)

    national_2020_df = clean_df(national_2020_df)
    arizona_2020_df = clean_df(arizona_2020_df)

    national_2019_df = clean_df(national_2019_df)
    arizona_2019_df = clean_df(arizona_2019_df)

def create_master_db_tables():
    global national_2023_df, national_2022_df, national_2021_df, national_2020_df, national_2019_df
    global arizona_2023_df, arizona_2022_df, arizona_2021_df, arizona_2020_df, arizona_2019_df
    # create national and state tables within an sqlite database
    create_national_table(national_2023_df, "2023")
    create_state_table(arizona_2023_df, "2023")

    create_national_table(national_2022_df, "2022")
    create_state_table(arizona_2022_df, "2022")

    create_national_table(national_2021_df, "2021")
    create_state_table(arizona_2021_df, "2021")

    create_national_table(national_2020_df, "2020")
    create_state_table(arizona_2020_df, "2020")

    create_national_table(national_2019_df, "2019")
    create_state_table(arizona_2019_df, "2019")

def remove_dups(limit_num):
    global national_2023_df, national_2022_df, national_2021_df, national_2020_df, national_2019_df
    global arizona_2023_df, arizona_2022_df, arizona_2021_df, arizona_2020_df, arizona_2019_df
    national_2023_df = sql_to_dataframe(run_query(f"SELECT * FROM national_data_2023 ORDER BY employment DESC LIMIT {limit_num};"))
    national_2023_df = national_2023_df.drop_duplicates(subset="occupation_title", keep="first")
    arizona_2023_df = sql_to_dataframe(run_query(f"SELECT * FROM arizona_state_data_2023 ORDER BY employment DESC LIMIT {limit_num};"))
    arizona_2023_df = arizona_2023_df.drop_duplicates(subset="occupation_title", keep="first")

    national_2022_df = sql_to_dataframe(run_query(f"SELECT * FROM national_data_2022 ORDER BY employment DESC LIMIT {limit_num};"))
    national_2022_df = national_2022_df.drop_duplicates(subset="occupation_title", keep="first")
    arizona_2022_df = sql_to_dataframe(run_query(f"SELECT * FROM arizona_state_data_2022 ORDER BY employment DESC LIMIT {limit_num};"))
    arizona_2022_df = arizona_2022_df.drop_duplicates(subset="occupation_title", keep="first")

    national_2021_df = sql_to_dataframe(run_query(f"SELECT * FROM national_data_2021 ORDER BY employment DESC LIMIT {limit_num};"))
    national_2021_df = national_2021_df.drop_duplicates(subset="occupation_title", keep="first")
    arizona_2021_df = sql_to_dataframe(run_query(f"SELECT * FROM arizona_state_data_2021 ORDER BY employment DESC LIMIT {limit_num};"))
    arizona_2021_df = arizona_2021_df.drop_duplicates(subset="occupation_title", keep="first")

    national_2020_df = sql_to_dataframe(run_query(f"SELECT * FROM national_data_2020 ORDER BY employment DESC LIMIT {limit_num};"))
    national_2020_df = national_2020_df.drop_duplicates(subset="occupation_title", keep="first")
    arizona_2020_df = sql_to_dataframe(run_query(f"SELECT * FROM arizona_state_data_2020 ORDER BY employment DESC LIMIT {limit_num};"))
    arizona_2020_df = arizona_2020_df.drop_duplicates(subset="occupation_title", keep="first")

    national_2019_df = sql_to_dataframe(run_query(f"SELECT * FROM national_data_2019 ORDER BY employment DESC LIMIT {limit_num};"))
    national_2019_df = national_2019_df.drop_duplicates(subset="occupation_title", keep="first")
    arizona_2019_df = sql_to_dataframe(run_query(f"SELECT * FROM arizona_state_data_2019 ORDER BY employment DESC LIMIT {limit_num};"))
    arizona_2019_df = arizona_2019_df.drop_duplicates(subset="occupation_title", keep="first")

def remove_occupations():
    # remove "Occupations" from the tables in the database
    tables = [f"national_data_{str(n)}" for n in range(2019,2024)]
    tables.extend([f"arizona_state_data_{str(n)}" for n in range(2019,2024)])

    for t in tables:
        q = f"DELETE FROM {t} WHERE occupation_title LIKE '%Occupations%'"
        run_commit_query(q)

def create_tables():
    global national_2023_df, national_2022_df, national_2021_df, national_2020_df, national_2019_df
    global arizona_2023_df, arizona_2022_df, arizona_2021_df, arizona_2020_df, arizona_2019_df
    national_2023_df = pd.read_html("may_2023_national_occupational_employment_and_wage_estimates.html")[0]
    national_2022_df = pd.read_html("may_2022_national_occupational_employment_and_wage_estimates.html")[0]
    national_2021_df = pd.read_html("may_2021_national_occupational_employment_and_wage_estimates.html")[0]
    national_2020_df = pd.read_html("may_2020_national_occupational_employment_and_wage_estimates.html")[0]
    national_2019_df = pd.read_html("may_2019_national_occupational_employment_and_wage_estimates.html")[0]

    arizona_2023_df = pd.read_html("arizona_may_2023_oews_state_occupational_employment_and_wage_estimates.html")[0]
    arizona_2022_df = pd.read_html("arizona_may_2022_oews_state_occupational_employment_and_wage_estimates.html")[0]
    arizona_2021_df = pd.read_html("arizona_may_2021_oews_state_occupational_employment_and_wage_estimates.html")[0]
    arizona_2020_df = pd.read_html("arizona_may_2020_oews_state_occupational_employment_and_wage_estimates.html")[0]
    arizona_2019_df = pd.read_html("arizona_may_2019_oes_state_occupational_employment_and_wage_estimates.html")[0]

In [4]:
create_tables()

In [5]:
create_clean_national_state_dfs()

In [6]:
create_master_db_tables()

In [7]:
remove_occupations()

In [8]:
remove_dups(100)

In [9]:
master_df = national_2023_df.copy()
master_df

,occupation_title,employment,mean_hourly_wage,annual_mean_wage
0,Material Moving Workers,7989980.0,19.05,39620.0
1,Retail Sales Workers,7655030.0,16.59,34520.0
2,Business Operations Specialists,7048350.0,42.33,88040.0
3,Laborers and Material Movers,6935980.0,18.46,38400.0
4,Food and Beverage Serving Workers,6893410.0,15.89,33060.0
...,...,...,...,...
95,"Shipping, Receiving, and Inventory Clerks",844120.0,20.54,42730.0
96,Management Analysts,838140.0,55.54,115530.0
97,Maids and Housekeeping Cleaners,836230.0,16.66,34650.0
98,"Lawyers, Judges, and Related Workers",792220.0,82.44,171480.0


In [35]:
master_df.columns = ["occupation_title", "2023_employment", "2023_mean_hourly_wage", "2023_annual_mean"]
master_df

,occupation_title,2023_employment,2023_mean_hourly_wage,2023_annual_mean
0,Material Moving Workers,7989980.0,19.05,39620.0
1,Retail Sales Workers,7655030.0,16.59,34520.0
2,Business Operations Specialists,7048350.0,42.33,88040.0
3,Laborers and Material Movers,6935980.0,18.46,38400.0
4,Food and Beverage Serving Workers,6893410.0,15.89,33060.0
...,...,...,...,...
95,"Shipping, Receiving, and Inventory Clerks",844120.0,20.54,42730.0
96,Management Analysts,838140.0,55.54,115530.0
97,Maids and Housekeeping Cleaners,836230.0,16.66,34650.0
98,"Lawyers, Judges, and Related Workers",792220.0,82.44,171480.0


In [ ]:
def name_me():
    global master_df
    col_names = ["occupation_title", "_employment", "_mean_hourly_wage", "_annual_mean_wage"]
    yrs = [f"{n}" for n in range(2019,2023)]
    temp = []
    for col in col_names[1:]:
        for n in range(2019,2023):
            n = str(n)
            temp.append(n + col)
    col_names = []

    for i in range(0,4):
        col_names.append(temp[i])
        col_names.append(temp[i+4])
        col_names.append(temp[i+8])

    master_df["2019_employment"] = national_2019_df["employment"]
    master_df["2020_employment"] = national_2020_df["employment"]
    master_df["2021_employment"] = national_2021_df["employment"]
    master_df["2022_employment"] = national_2022_df["employment"]

    master_df["2019_mean_hourly_wage"] = national_2019_df["mean_hourly_wage"]
    master_df["2020_mean_hourly_wage"] = national_2020_df["mean_hourly_wage"]
    master_df["2021_mean_hourly_wage"] = national_2021_df["mean_hourly_wage"]
    master_df["2022_mean_hourly_wage"] = national_2022_df["mean_hourly_wage"]

    master_df["2019_annual_mean_wage"] = national_2019_df["annual_mean_wage"]
    master_df["2020_annual_mean_wage"] = national_2020_df["annual_mean_wage"]
    master_df["2021_annual_mean_wage"] = national_2021_df["annual_mean_wage"]
    master_df["2022_annual_mean_wage"] = national_2022_df["annual_mean_wage"]

In [29]:
name_me()

['2019_employment',
 '2019_mean_hourly_wage',
 '2019_annual_mean_wage',
 '2020_employment',
 '2020_mean_hourly_wage',
 '2020_annual_mean_wage',
 '2021_employment',
 '2021_mean_hourly_wage',
 '2021_annual_mean_wage',
 '2022_employment',
 '2022_mean_hourly_wage',
 '2022_annual_mean_wage']

In [23]:
national_2023_df

,occupation_title,employment,mean_hourly_wage,annual_mean_wage
0,Material Moving Workers,7989980.0,19.05,39620.0
1,Retail Sales Workers,7655030.0,16.59,34520.0
2,Business Operations Specialists,7048350.0,42.33,88040.0
3,Laborers and Material Movers,6935980.0,18.46,38400.0
4,Food and Beverage Serving Workers,6893410.0,15.89,33060.0
...,...,...,...,...
95,"Shipping, Receiving, and Inventory Clerks",844120.0,20.54,42730.0
96,Management Analysts,838140.0,55.54,115530.0
97,Maids and Housekeeping Cleaners,836230.0,16.66,34650.0
98,"Lawyers, Judges, and Related Workers",792220.0,82.44,171480.0


In [24]:
national_2022_df

,occupation_title,employment,mean_hourly_wage,annual_mean_wage
0,Material Moving Workers,7896560.0,17.92,37270.0
1,Retail Sales Workers,7584940.0,15.62,32500.0
2,Laborers and Material Movers,6840530.0,17.33,36050.0
3,Business Operations Specialists,6660810.0,40.04,83280.0
4,Food and Beverage Serving Workers,6309200.0,14.69,30540.0
...,...,...,...,...
95,Food Processing Workers,807880.0,17.45,36300.0
96,Market Research Analysts and Marketing Special...,798620.0,37.92,78880.0
97,Industrial Truck and Tractor Operators,780890.0,20.79,43240.0
98,Maids and Housekeeping Cleaners,771390.0,15.35,31920.0


In [25]:
arizona_2019_df

,occupation_title,employment,mean_hourly_wage,annual_mean_wage
0,Customer Service Representatives,105000.0,17.02,35400.0
1,Retail Salespersons,89290.0,13.90,28910.0
2,Home Health and Personal Care Aides,66890.0,12.53,26050.0
3,Cashiers,65170.0,12.52,26030.0
4,Fast Food and Counter Workers,63280.0,12.26,25510.0
...,...,...,...,...
95,"Medical Dosimetrists, Medical Records Speciali...",7880.0,22.38,46550.0
96,"Production, Planning, and Expediting Clerks",7810.0,22.97,47780.0
97,"Child, Family, and School Social Workers",7630.0,19.91,41420.0
98,Licensed Practical and Licensed Vocational Nurses,7600.0,26.10,54290.0


In [26]:
national_df = sql_to_dataframe(run_query("SELECT occupation_title, SUM(employment) as total_employment FROM national_data WHERE mean_hourly_wage > 10 GROUP BY occupation_title ORDER BY total_employment DESC;"))
state_df = sql_to_dataframe(run_query("SELECT * FROM state_data WHERE mean_hourly_wage > 10 ORDER BY employment DESC LIMIT 20;"))

national_df["percentage_employed"] = (national_df["total_employment"] / national_df["total_employment"].sum()) * 100
combined_df = combine_national_state_df(national_df, state_df)

OperationalError: no such table: national_data

In [ ]:
make_plot(x = combined_df["occupation_title"],
    y = combined_df["national_employment"].astype(int),
    title = "national employment",
    xlabel="occupation",
    ylabel="employment")

make_plot(x = combined_df["occupation_title"],
    y = combined_df["arizona_state_employment"].astype(int),
    title = "arizona state employment",
    xlabel="occupation",
    ylabel="employment")

plt.show()

In [ ]:
combined_df["national_employment_percentage"] = national_df["percentage_employed"]

In [ ]:
"May 2022 National Occupational Employment and Wage Estimates.html".lower().replace(" ", "_")

In [ ]:
https://www.bls.gov/oes/2020/may/oes_az.htm